author: leezeeyee   
date: 2021/4/13  
link: [github](https://github.com/easilylazy/pattern-recognition) 

In [2]:
import pandas as pd


In [48]:
import random
import numpy as np

## Lecture 9编程作业
给定IRIS数据集做如下题目实验，该数据集有三类目标，每个类别有50个样本，每个样本有四维特征。实验时每个类别随机选30个样本进行训练，另外20个样本用于测试。  
1. 以感知器算法为基础分类算法，编写一个OVO多类分类器算法，对上述数据集进行实验，分析结果。    
2. 编写Softmax算法实现多类别分类，对上述数据集进行实验，分析结果。


In [6]:
irisData=pd.read_csv('iris.csv')

In [7]:
irisData.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [12]:
irisData['Species'][60]

'versicolor'

## types

In [13]:
setosas=irisData[irisData['Species']=='setosa']
versicolors=irisData[irisData['Species']=='versicolor']
virginicas=irisData[irisData['Species']=='virginica']

In [14]:
setosas.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [15]:
versicolors.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
50,51,7.0,3.2,4.7,1.4,versicolor
51,52,6.4,3.2,4.5,1.5,versicolor
52,53,6.9,3.1,4.9,1.5,versicolor
53,54,5.5,2.3,4.0,1.3,versicolor
54,55,6.5,2.8,4.6,1.5,versicolor


In [16]:
virginicas.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
100,101,6.3,3.3,6.0,2.5,virginica
101,102,5.8,2.7,5.1,1.9,virginica
102,103,7.1,3.0,5.9,2.1,virginica
103,104,6.3,2.9,5.6,1.8,virginica
104,105,6.5,3.0,5.8,2.2,virginica


## sample for train

In [19]:

def some(x, n):
    return x[random.sample(x.index, n)]

In [28]:
virginicas.index

Int64Index([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
            113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
            126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
            139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
           dtype='int64')

In [192]:
trainSetosas=setosas.sample(frac=0.6)
trainVersi=versicolors.sample(frac=0.6)
trainVirgin=virginicas.sample(frac=0.6)
testSetosas=setosas.sample(frac=0.4)
testVersi=versicolors.sample(frac=0.4)
testVirgin=virginicas.sample(frac=0.4)

In [240]:
wholeData=[]
for data in [setosas,versicolors,virginicas]:
    values=data[['Sepal.Length',	'Sepal.Width',	'Petal.Length',	'Petal.Width']]
    wholeData.append(np.asarray(values.iloc[:,:]))

In [214]:
trainData=[]
for train in [trainSetosas,trainVersi,trainVirgin]:
    values=train[['Sepal.Length',	'Sepal.Width',	'Petal.Length',	'Petal.Width']]
    trainData.append(np.asarray(values.iloc[:,:]))
testData=[]
for test in [testSetosas,testVersi,testVirgin]:
    values=test[['Sepal.Length',	'Sepal.Width',	'Petal.Length',	'Petal.Width']]
    testData.append(np.asarray(values.iloc[:,:]))

## Train

In [61]:
def addSingleBias(X):
    N=X.shape[0]
    X=np.reshape(X,[N,1])
    bias_vector=np.ones([1,1])
    return np.row_stack((bias_vector,X))
def addBias(X):
    '''
    为维数为N,d的向量X添加偏置向量
    Args:
        X(numpy.ndarray):shape为[N,d]
    Returns:
        numpy.ndarray:shape为[N,d+1]，且第一列列向量全为1
    '''
    
    try:
        N=X.shape[0]
        d=X.shape[1]
        if d == 1:
            return addSingleBias(X)
        bias_vector=np.ones([N,1])
        return np.column_stack((bias_vector,X))
        
    except:
        return addSingleBias(X)

In [216]:
trainData=np.asarray(trainData)
testData=np.asarray(testData)

In [195]:
trainData[0][:5]

array([[5.2, 3.4, 1.4, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.1, 3.8, 1.9, 0.4],
       [5. , 3.5, 1.6, 0.6],
       [4.6, 3.2, 1.4, 0.2]])

# 整体

In [227]:
def Perce2(TrainX_sh,TrainY_sh):
    learning_rate=1
    minErr=60
    W=np.zeros([5,1])
    for iteration in range(20):
        for i in range(len(TrainX_sh)):
            X=TrainX_sh[i].reshape([5,1])
            y=TrainY_sh[i]
            if np.dot(W.transpose(),X)*y<=0:
                W_temp=W+learning_rate*y*X
                res=np.dot(TrainX_sh,W)*TrainY_sh
                err1=len(res[res<=0])
                res=np.dot(TrainX_sh,W_temp)*TrainY_sh
                err2=len(res[res<=0])
                # if err1>=err2:
                W=W_temp
            res=np.dot(TrainX_sh,W)*TrainY_sh
            err1=len(res[res<=0])
            if err1<minErr:
                    minErr=err1
                    # print('******************')
                    # print(W)
                    W_end=W
                    # print(err1) 
        if minErr==0:
            break
    return W_end
        

## train for weights

In [228]:
# 1 for left and -1 for right
rounds=[[0,1],[0,2],[1,2]]
# length of train data for 1 round
length=60
# Ws
Ws=[]
for round in rounds:
    print(round)
    left=round[0]
    right=round[1]
    # load data
    X1=addBias(trainData[left])
    X2=addBias(trainData[right])
    TrainX=np.row_stack([X1,X2])
    TrainY=np.r_[np.ones([len(X1),1]),-1*np.ones([len(X2),1])]
    X1_t=addBias(testData[left])
    X2_t=addBias(testData[right])
    TestX=np.r_[X1_t,X2_t]
    TestY=np.r_[np.ones([len(X1_t),1]),-1*np.ones([len(X2_t),1])]
   
    # shuffle
    numArray=np.arange(length)
    np.random.shuffle(numArray)

    TrainX_sh=TrainX[numArray]
    TrainY_sh=TrainY[numArray]
    # train
    W_end=Perce2(TrainX_sh,TrainY_sh)
    Ws.append(W_end)
    # test
    testRes=np.dot(TestX,W_end)*TestY
    err=len(testRes[testRes<=0])
    print(err)

[0, 1]
0
[0, 2]
0
[1, 2]
0


## test OvO

In [243]:
for test_i in range(len(testData)):
    print(test_i)
    X_t=addBias(testData[test_i])
    # X_t=addBias(wholeData[test_i])
    for X in X_t:
        records=np.zeros(len(testData))
        for i in range(len(rounds)):
            round=rounds[i]
            # print(round)
            left=round[0]
            right=round[1]
            W=Ws[i]
            # print(np.dot(X,W))
            res=np.dot(X,W)
            if res>0:
                records[left]+=1
            else:
                records[right]+=1
        # print(records)
        # print(np.argmax(records))
        if test_i!=np.argmax(records):
            print(W)
            print(X)  


0
1
2


# 单轮

## shuffle data

In [196]:
length=60
numArray=np.arange(length)
np.random.shuffle(numArray)

In [197]:
X1=addBias(trainData[0])
X2=addBias(trainData[1])
TrainX=np.row_stack([X1,X2])
TrainY=np.r_[np.ones([len(X1),1]),-1*np.ones([len(X2),1])]
print(TrainX[25:35])
print(TrainY[25:35])
TrainX_sh=TrainX[numArray]
TrainY_sh=TrainY[numArray]
if np.not_equal(TrainX[numArray[0]],TrainX_sh[0]).any():
    print('error!')

[[1.  5.  3.4 1.6 0.4]
 [1.  4.6 3.4 1.4 0.3]
 [1.  4.4 3.  1.3 0.2]
 [1.  5.2 3.5 1.5 0.2]
 [1.  5.4 3.9 1.7 0.4]
 [1.  6.2 2.2 4.5 1.5]
 [1.  6.1 3.  4.6 1.4]
 [1.  6.9 3.1 4.9 1.5]
 [1.  6.  2.9 4.5 1.5]
 [1.  5.5 2.5 4.  1.3]]
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]]


In [198]:
learning_rate=0.1

def Perce(X,y,w_ini,epoch=5,step=False,separable=True):
    leastWrong=len(y)
    totalMistake=len(y)
    wT=w_ini.transpose()
    for t in range(epoch):
        wrong=0
        print(t)
        for i in range(len(y)):
            ynE=np.dot(wT,np.transpose(X[i]))
            if ynE*y[i]<=0:
                # represent error, need to correct w
                wT1=wT+y[i]*X[i]
                if separable :
                    wrong+=1
                    wT=wT1
                    if step:
                        yield wT,wrong
                else:
                    res=np.multiply(np.dot(X,wT1.transpose()),y) 
                    mistake=len(res[res<=0])
                    print(mistake)
                    if mistake<totalMistake:
                        wT=wT1
                        totalMistake=mistake
                        yield wT,mistake
        if separable and wrong<leastWrong:
            leastWrong=wrong
            print('wrong time: ',end='')
            print(wrong)
            print(wT)
            yield wT,wrong
            if wrong==0:
                break

W=np.zeros([5,1])
for i in Perce(TrainX_sh,TrainY_sh,w_ini=W,epoch=5,step=False,separable=False):
    print(i)

In [153]:
TrainY_sh[i]

array([1.])

In [201]:
learning_rate=1
minErr=60
W=np.zeros([5,1])
for iteration in range(20):
    for i in range(len(TrainX_sh)):
        # print(X1[i])
        X=TrainX_sh[i].reshape([5,1])
        # print(X)
        y=TrainY_sh[i]
        # print(X)
        # print(y)
        if np.dot(W.transpose(),X)*y<=0:
            W_temp=W+learning_rate*y*X
            # print(W_temp)
            res=np.dot(TrainX_sh,W)*TrainY_sh
            err1=len(res[res<=0])
            res=np.dot(TrainX_sh,W_temp)*TrainY_sh
            err2=len(res[res<=0])
            
            # if err1>=err2:
            W=W_temp
            # print(err1)
            # print(err2) 
        
        res=np.dot(TrainX_sh,W)*TrainY_sh
        err1=len(res[res<=0])
        # print(err1)
        if err1<minErr:
                minErr=err1
                print('******************')
                print(W)
                W_end=W
                print(err1)        

******************
[[1. ]
 [5.1]
 [3.8]
 [1.5]
 [0.3]]
30
******************
[[ 1. ]
 [ 0.8]
 [ 6.1]
 [-8.2]
 [-3.3]]
0


## validation

In [218]:
# W_end=np.asarray([[ 1.,   0.7 , 3.9 ,-8.2 ,-2.7]])
trainRes=np.dot(TrainX_sh,W_end)*TrainY_sh
err=len(trainRes[trainRes<=0])
print(err)
TestX=np.row_stack([addBias(testData[0]),addBias(testData[1])])
TestY=np.r_[np.ones([len(testData[0]),1]),-1*np.ones([len(testData[0]),1])]
testRes=np.dot(TestX,W_end)*TestY
err=len(testRes[testRes<=0])
print(err)

0
0


In [219]:
print(testData[0][:3])
print(testData[1][:3])

[[5.4 3.7 1.5 0.2]
 [4.8 3.1 1.6 0.2]
 [4.7 3.2 1.3 0.2]]
[[6.  3.4 4.5 1.6]
 [5.6 2.5 3.9 1.1]
 [6.4 2.9 4.3 1.3]]


In [181]:
W_end=np.asarray([[1. , 4.5, 2.3, 1.3, 0.3]])
np.dot(TrainX_sh,W_end.transpose())*TrainY_sh


array([[ 28.32],
       [-41.36],
       [-38.81],
       [-41.55],
       [-44.84],
       [ 31.43],
       [ 35.37],
       [ 32.54],
       [ 34.46],
       [-39.37],
       [-42.15],
       [ 34.53],
       [ 33.33],
       [-46.  ],
       [-40.26],
       [ 34.1 ],
       [-33.16],
       [-39.4 ],
       [-38.56],
       [-43.46],
       [ 32.95],
       [-40.75],
       [-38.5 ],
       [-41.32],
       [ 31.83],
       [ 36.08],
       [-37.35],
       [-42.45],
       [-41.65],
       [ 37.69],
       [ 29.35],
       [-44.42],
       [-36.54],
       [ 33.88],
       [ 32.97],
       [-46.39],
       [-41.92],
       [-43.74],
       [ 28.71],
       [-38.94],
       [ 31.34],
       [ 29.91],
       [-40.97],
       [ 29.45],
       [ 32.16],
       [ 33.9 ],
       [ 30.94],
       [ 32.19],
       [ 33.66],
       [ 32.56],
       [-36.38],
       [ 33.52],
       [-33.38],
       [ 33.33],
       [ 37.92],
       [ 33.18],
       [-38.26],
       [-38.64],
       [-39.33

In [51]:
foo=np.asarray(values.iloc[:,:])